In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
import string_constants as sc
import reading_util as ru
import writing_util as wu
import dataset_util as du
import parameter as pa
import quantile_regression_dnn as qr
import data_sample as ds
import plotting_util as pu
import plotting_util_analysis as pua
import flat_cut as fc
import selector as sr
import discriminator as dm

Welcome to JupyROOT 6.12/07


In [3]:
def make_run_path( strategy, quantile, selector ):
    return os.path.join( strategy.path_str,  quantile.path_str, selector.path_str )

def make_run_title( strategy, quantile, selector ):
    return strategy.title_str + ' ' + quantile.title_str + ' ' + selector.title_str

# runtime parameters

In [4]:
# share of total qcd signal region data for QR training (rest for GOF test)
QR_train_share = 0.25 #0.2
# constant fraction wrt dijet mass of BG accepted
quantile = pa.quantile_dict['q10'] # q1 = 1%
# strategy for combining loss-jet-1 and loss-jet-2
strategy = pa.loss_strategy_dict['s5'] # s5 = L1 & L2
# discriminator accepted - rejected
selector = pa.selector_dict['qr_full']
# make path
pathstr = make_run_path( strategy, quantile, selector )

In [5]:
date = datetime.date.today().strftime("%m%d")

# read in data (qcd and signal)

In [6]:
SM_sample = 'qcdSigAll'
qcd_path = os.path.join( sc.concat_result_dir, sc.sample_loc[SM_sample] + sc.concat_result_suffix )
qcd_data_total = ru.read_results_to_dataframe( qcd_path )

## split qcd data into train and test set 

In [7]:
qcd_data_train, qcd_data_test = du.split_dataset_train_test( qcd_data_total, QR_train_share )

## make qcd_train and qcd_test data samples

In [8]:
qcd_sample_train = ds.DataSample( SM_sample+'_train', qcd_data_train )
qcd_sample_test = ds.DataSample( SM_sample, qcd_data_test )

# Discriminator

In [9]:
discriminator = dm.Discriminator( quantile, strategy, selector )
discriminator.train_selector( qcd_sample_train )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               200       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 30,601
Trainable params: 30,601
Non-trainable params: 0
_________________________________________________________________
Train 

2020-05-20 14:06:35.527990: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


In [11]:
model_base_path = os.path.join('models',pathstr)
print(model_base_path)
!mkdir -p $model_base_path
run = 6
model_path = 'model_'+date+'_run_'+str(run)+'.h5'
print(os.path.join(model_base_path, model_path))
discriminator.save_selector(os.path.join(model_base_path, model_path))

models/minl1l2_loss/qu_10pct/QR_full
models/minl1l2_loss/qu_10pct/QR_full/model_0520_run_6.h5
